In [4]:
from typing import *
import os
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tqdm.notebook import tqdm
from nptyping import NDArray, Shape, Int, Float
from rich import print as rprint

sns.set_style('whitegrid')
colors = ['#de3838', '#007bc3', '#ffd12a']
markers = ['o', 'x', ',']
%config InlineBackend.figure_formats = ['svg']

cmap = sns.diverging_palette(255, 0, as_cmap=True)  # カラーパレットの定義
DATA_DIR = Path()/'..'/'..'/'data'
DF_DIR = DATA_DIR/'dataframes'

In [8]:
# 11

df_uriage_data = pd.read_csv(DATA_DIR/'uriage.csv')
df_kokyaku_data = pd.read_excel(DATA_DIR/'kokyaku_daicho.xlsx')
display(df_uriage_data.head(2))
display(df_kokyaku_data.head(2))

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二


,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782


In [11]:
# 14
df_uriage_data = pd.read_csv(DATA_DIR/'uriage.csv')

unique = pd.unique(df_uriage_data['item_name'])
print(f'item names: {unique}')
print(f'unique size: {len(unique)}')

df_uriage_data['item_name'] = df_uriage_data['item_name'].str.upper()
df_uriage_data['item_name'] = df_uriage_data['item_name'].str.replace('　', '')
df_uriage_data['item_name'] = df_uriage_data['item_name'].str.replace(' ', '')
display(df_uriage_data)

unique = pd.unique(df_uriage_data["item_name"])
print(f'item names: {unique}')
print(f'unique size: {len(unique)}')

df_uriage_data.to_pickle(DF_DIR/'uriage_data_k14.pkl')

item names: ['商品A' '商 品 S' '商 品 a' '商品Z' '商品a' '商品S' '商品 a' '商品V' '商品O' '商 品U' '商品L'
 '商  品V' '商 品O' '商品C' '商品I' '商品r' '商品X' '商品 g' '商品R' '商品P' '商品Q' '商品y'
 '商品 A' '商品N' '商品W' '商 品E' '商品K' '商品B' '商品F' '商 品s' '  商品W' ' 商 品 n'
 '商 品F' '商品D' '商品M' '商品Y' '商品U' '商品H' '商品T' '商品J' '商  品O' '商品E' '商  品Q'
 ' 商品S' ' 商品M' '商  品T' '商品G' '商 品G' ' 商品P' ' 商品E' '商 品N' '商 品Y' '商品 J'
 '商品 V' '商品 K' '商 品V' '商 品D' '商 品A' '商品 F' '商品 H' '商 品K' '商 品T' '商品 X'
 '商品 Q' '商 品X' '商 品H' '商 品C' '商品 B' '商品 O' '商品 T' '商品v' '商品p' '商品i' '商品 w'
 '商 品 s' '商 品 q' '商品s' '商品l' '商品t' '商品k' '商品g' '商品o' '商品 R' '商品 S' '商 品M'
 '商品j' '商品d' '商品 I' '商品 E' '商品 o' '商品c' '商品 v' '商品e' '商品x' '商 品I' ' 商品W'
 ' 商品X' '商品 M' '商 品P']
unique size: 99


,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商品S,NaN,浅田賢二
2,2019-05-11 19:42:07,商品A,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品A,NaN,平田鉄二
...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司
2996,2019-03-29 11:14:05,商品Q,NaN,尾形小雁
2997,2019-07-14 12:56:49,商品H,NaN,芦田博之


item names: ['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
unique size: 26


In [28]:
# 15

df_uriage_data = pd.read_pickle(DF_DIR/'uriage_data_k14.pkl')

display(df_uriage_data.isnull().any(axis=0))
is_null = df_uriage_data['item_price'].isnull()

price_null_item_names = df_uriage_data.loc[is_null, 'item_name'].unique()
for item_name in price_null_item_names:
    max_price = df_uriage_data.loc[(~is_null) & (df_uriage_data['item_name'] == item_name), 'item_price'].max()
    df_uriage_data.loc[(is_null) & (df_uriage_data['item_name'] == item_name), 'item_price'] = max_price
display(df_uriage_data)

# 検証
display(df_uriage_data.isnull().any(axis=0))
unique_items = df_uriage_data['item_name'].sort_values().unique()
for item_name in unique_items:
    max_price = df_uriage_data.loc[df_uriage_data['item_name'] == item_name, 'item_price'].max()
    min_price = df_uriage_data.loc[df_uriage_data['item_name'] == item_name, 'item_price'].min(skipna=False)
    print(f'{item_name} の最大額: {max_price}, の最小額: {min_price}, is equal: {max_price==min_price}')

df_uriage_data.to_pickle(DF_DIR/'uriage_data_k15.pkl')

purchase_date    False
item_name        False
item_price        True
customer_name    False
dtype: bool

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二
2,2019-05-11 19:42:07,商品A,100.0,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二
...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司
2996,2019-03-29 11:14:05,商品Q,1700.0,尾形小雁
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之


purchase_date    False
item_name        False
item_price       False
customer_name    False
dtype: bool

商品A の最大額: 100.0, の最小額: 100.0, is equal: True
商品B の最大額: 200.0, の最小額: 200.0, is equal: True
商品C の最大額: 300.0, の最小額: 300.0, is equal: True
商品D の最大額: 400.0, の最小額: 400.0, is equal: True
商品E の最大額: 500.0, の最小額: 500.0, is equal: True
商品F の最大額: 600.0, の最小額: 600.0, is equal: True
商品G の最大額: 700.0, の最小額: 700.0, is equal: True
商品H の最大額: 800.0, の最小額: 800.0, is equal: True
商品I の最大額: 900.0, の最小額: 900.0, is equal: True
商品J の最大額: 1000.0, の最小額: 1000.0, is equal: True
商品K の最大額: 1100.0, の最小額: 1100.0, is equal: True
商品L の最大額: 1200.0, の最小額: 1200.0, is equal: True
商品M の最大額: 1300.0, の最小額: 1300.0, is equal: True
商品N の最大額: 1400.0, の最小額: 1400.0, is equal: True
商品O の最大額: 1500.0, の最小額: 1500.0, is equal: True
商品P の最大額: 1600.0, の最小額: 1600.0, is equal: True
商品Q の最大額: 1700.0, の最小額: 1700.0, is equal: True
商品R の最大額: 1800.0, の最小額: 1800.0, is equal: True
商品S の最大額: 1900.0, の最小額: 1900.0, is equal: True
商品T の最大額: 2000.0, の最小額: 2000.0, is equal: True
商品U の最大額: 2100.0, の最小額: 2100.0, is equal: True
商品V の最大額: 2200.0, の最小額: 2200.0,

In [30]:
# 16

df_kokyaku_data = pd.read_excel(DATA_DIR/'kokyaku_daicho.xlsx')

df_kokyaku_data['顧客名'] = df_kokyaku_data['顧客名'].str.replace('　', '')
df_kokyaku_data['顧客名'] = df_kokyaku_data['顧客名'].str.replace(' ', '')
display(df_kokyaku_data)

df_kokyaku_data.to_pickle(DF_DIR/'kokyaku_data_k16.pkl')

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017/06/20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018/06/20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017/04/29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019/04/19


In [57]:
# 17

df_kokyaku_data = pd.read_pickle(DF_DIR/'kokyaku_data_k16.pkl')

is_degit = df_kokyaku_data['登録日'].astype('str').str.isdigit()
print(f'length of days as degit: {is_degit.sum()}')


# 1. Eｘcel上の最小日付は1900/01/01であり、"1"始まりである
# 2. Excel上の日付仕様では、本来うるう年ではない1900/02/29を有効な日付として加算している
# 1, 2よりExcel値から 2 を引く
from_degit = pd.to_timedelta(df_kokyaku_data.loc[is_degit, '登録日'].astype('float') - 2, unit='D') + pd.to_datetime('1900/1/1') 
display(from_degit.head())

from_string = pd.to_datetime(df_kokyaku_data.loc[~is_degit, '登録日'])
display(from_string.head())

df_kokyaku_data['登録日'] = pd.concat([from_degit, from_string])
display(df_kokyaku_data)

df_kokyaku_data['登録年月'] = df_kokyaku_data['登録日'].dt.strftime('%Y%m')

display(df_kokyaku_data.groupby('登録年月').count()[['顧客名']])

# 検証
is_degit = df_kokyaku_data['登録日'].astype('str').str.isdigit()
print(f'length of days as degit: {is_degit.sum()}')

df_kokyaku_data.to_pickle(DF_DIR/'kokyaku_data_k17.pkl')

length of days as degit: 22


1    2017-02-16
3    2017-05-17
4    2018-01-27
21   2017-07-04
27   2017-06-15
Name: 登録日, dtype: datetime64[ns]

0   2018-01-04
2   2018-01-07
5   2017-06-20
6   2018-06-11
7   2017-05-19
Name: 登録日, dtype: datetime64[ns]

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


,顧客名
登録年月,
201701,15
201702,11
201703,14
201704,15
201705,14
201706,13
201707,17
201801,13
201802,15


length of days as degit: 0


In [61]:
# 18

df_uriage_data = pd.read_pickle(DF_DIR/'uriage_data_k15.pkl')
df_kokyaku_data = pd.read_pickle(DF_DIR/'kokyaku_data_k17.pkl')

df_uriage_with_kokyaku = pd.merge(df_uriage_data, df_kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
df_uriage_with_kokyaku = df_uriage_with_kokyaku.drop('customer_name', axis=1)
display(df_uriage_with_kokyaku)

df_uriage_with_kokyaku.to_pickle(DF_DIR/'uriage_with_kokyaku_k18.pkl')

,purchase_date,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13,201807


In [70]:
# 19
df_uriage_with_kokyaku = pd.read_pickle(DF_DIR/'uriage_with_kokyaku_k18.pkl')

df_uriage_with_kokyaku['purchase_date'] = pd.to_datetime(df_uriage_with_kokyaku['purchase_date'])
df_uriage_with_kokyaku['purchase_month'] = df_uriage_with_kokyaku['purchase_date'].dt.strftime('%Y%m')
columns = ['purchase_date', 'purchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日', '登録年月']
df_uriage_with_kokyaku = df_uriage_with_kokyaku[columns]
display(df_uriage_with_kokyaku.head(2))

df_uriage_with_kokyaku.to_pickle(DF_DIR/'uriage_with_kokyaku_k19.pkl')

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804


In [76]:
# 20

df_uriage_with_kokyaku = pd.read_pickle(DF_DIR/'uriage_with_kokyaku_k19.pkl')

by_item = df_uriage_with_kokyaku.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
display(by_item)

by_price = df_uriage_with_kokyaku.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
display(by_price)

by_customer = df_uriage_with_kokyaku.pivot_table(index='purchase_month', columns='顧客名', aggfunc='size', fill_value=0)
display(by_customer)

by_region = df_uriage_with_kokyaku.pivot_table(index='purchase_month', columns='地域', aggfunc='size', fill_value=0)
display(by_region)


item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54
